In [1]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pyautogui
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#position of side bar
position_x = 270
position_y = 400

In [3]:
province_list = ["Drenthe", "Flevoland", "Friesland", "Gelderland", "Groningen", "Limburg", "Noord-Brabant", "Noord-Holland", "Overijssel", "Utrecht", "Zeeland", "Zuid-Holland"]
query = "electrician"
country = "Netherlands"
URL_list = []

for name in province_list:
    url = f'https://www.google.com/maps/search/{query}+in+{name}+{country}/'
    URL_list.append(url)

In [4]:
driver = webdriver.Chrome()

In [5]:
#Load all the dynamic content by scrolling to the bottom of the page
def scroller():
    stop_text = "Je hebt het einde van de lijst bereikt."
    test = []
    timeout = time.time() + 60*5
    while test == []:
        if (time.time() > timeout):
            #refresh the page if it takes too long
            driver.refresh()
        pyautogui.moveTo(position_x, position_y)
        pyautogui.scroll(-1000)
        pyautogui.moveTo(position_x, position_y)
        #check if the end of the page is reached
        test = driver.find_elements(By.XPATH, "//span[contains(text(), 'Je hebt het einde van de lijst bereikt.')]")

In [6]:
#search element class 'Nv2PK THOPZb CpccDe '

In [7]:
#pre condition: driver is on the google maps page with the search results (fully loaded)
def find_map_url_google(driver):
    found_map_url = driver.find_elements(By.XPATH,"//a[@href]")
    found_map_url_google = []
    for elem in found_map_url:
        #make sure its a google maps url
        if "https://www.google.com/maps/place/" in elem.get_attribute("href"):
            found_map_url_google.append(elem.get_attribute("href"))
    return found_map_url_google

In [8]:
def extract_info(map_url, driver):
    try:
        # Create a new tab
        driver.switch_to.new_window('tab')
        # Load the URL
        driver.get(map_url)
        # Wait for the page to load
        driver.implicitly_wait(1)

        # Initialize variables
        name = ""
        address = ""
        phone = ""
        website = ""

        # Attempt to extract information
        try:
            name_raw = driver.find_element(By.CSS_SELECTOR, "[aria-label^='Informatie voor']")
            name = name_raw.get_attribute("aria-label")
            name = name.replace("Informatie voor ", "")
        except NoSuchElementException:
            pass

        try:
            address_raw = driver.find_element(By.CSS_SELECTOR, "[aria-label^='Adres']")
            address = address_raw.get_attribute("aria-label")
            address = address.replace("Adres: ", "")
        except NoSuchElementException:
            pass

        try:
            phone_raw = driver.find_element(By.CSS_SELECTOR, "[aria-label^='Telefoon']")
            phone = phone_raw.get_attribute("aria-label")
            phone = phone.replace("Telefoon: ", "")
        except NoSuchElementException:
            pass

        try:
            website_raw = driver.find_element(By.CSS_SELECTOR, "[data-item-id^='authority']")
            website = website_raw.get_attribute("href")
        except NoSuchElementException:
            pass

        # Close the tab
        driver.close()
        # Switch to the previous tab
        driver.switch_to.window(driver.window_handles[0])

        return name, address, phone, website

    except Exception as e:
        # Handle any exceptions that occur during the extraction
        print(f"An error occurred: {e}")
        return None, None, None, None

In [9]:
electrician_df = pd.DataFrame(columns=["Name", "Address", "Phone", "Website"])

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
from IPython.display import clear_output

In [12]:
current_index = 0
for url in URL_list:
    driver.get(url)
    time.sleep(5)
    scroller()
    found_map_url_google = find_map_url_google(driver)
    for map_url in found_map_url_google:
        name, address, phone, website = extract_info(map_url, driver)
        print("adding to dataframe")
        print(name, address, phone, website)
        entry = {"Name": name, "Address": address, "Phone": phone, "Website": website}
        electrician_df = electrician_df.append(entry, ignore_index=True)
    #clear the print
    clear_output()
    current_index += 1

In [15]:
electrician_df.to_csv("./Data/googlemapselectrician/electrician_df.csv")

In [17]:
electrician_df

,Name,Address,Phone,Website
0,E. Remmelts Elektrotechniek,"Binnenes 71, 9407 CX Assen, Nederland",+31 6 30888249,http://www.remmeltselektro.nl/
1,Zeemanelektro.nl B.V.,"Jachthoorn 19, 7833 CX Nieuw-Amsterdam, Nederl...",+31 591 858 156,https://www.zeemanelektro.nl/
2,Kiers Elektro,,+31 6 37612395,http://www.kierselektro.nl/
3,Meints Techniek,"Anreperstraat 152, 9404 LJ Assen, Nederland",+31 6 39300450,https://www.meintstechniek.nl/
4,Elektrotechnisch Installatiebedrijf Middel,"Waardeel 2, 9531 EC Borger, Nederland",+31 6 15468247,http://www.middelelektro.nl/
...,...,...,...,...
1153,Yunex Traffic B.V.,,"Telefoon,Telefoonnummer van plaats toevoegen",https://nl.yunextraffic.com/
1154,Ster solar,,+31 6 43991507,https://www.ster-solar.nl/
1155,JS Elektrotechnisch Installatiebureau,,+31 70 212 3933,
1156,Elektricien Amsterdam 24/7,"Bankwerkerij 39, 1021 NS Amsterdam, Nederland",+31 85 019 0036,https://elektricienamsterdam247.nl/
